In [34]:
import pandas as pd
import numpy as np
import utils
import matplotlib.pyplot as plt
import matplotlib
import sklearn as sk

from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, mutual_info_regression, RFE
from sklearn.linear_model import LinearRegression, LassoCV
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.manifold import TSNE, SpectralEmbedding
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.svm import LinearSVC
from sklearn.metrics import adjusted_rand_score, adjusted_mutual_info_score, homogeneity_completeness_v_measure, silhouette_score, mean_squared_error
from sklearn.metrics import classification_report, confusion_matrix, f1_score

plt.style.use("ggplot")
pd.set_option('display.max_rows', 500)

In [28]:
mdf = pd.read_csv('M_athlete_results.csv', low_memory=False)
fdf = pd.read_csv('F_athlete_results.csv', low_memory=False)

In [29]:
# String marks to float times in seconds
mdf['Time'] = [utils.Time(m) if (e in utils.event_classification()['Track'] and utils.ismark(m)) else None for e, m in zip(mdf['Event'], mdf['Mark'])]
fdf['Time'] = [utils.Time(m) if (e in utils.event_classification()['Track'] and utils.ismark(m)) else None for e, m in zip(fdf['Event'], fdf['Mark'])]

In [30]:
# String events to float distance in meters
mdf['Distance'] = [utils.string_to_distance(e) if e in utils.event_classification()['Track'] else None for e in mdf['Event']]
fdf['Distance'] = [utils.string_to_distance(e) if e in utils.event_classification()['Track'] else None for e in fdf['Event']]

In [31]:
mdf_dataframe = pd.DataFrame(mdf)
fdf_dataframe = pd.DataFrame(fdf)

csm_m = mdf.loc[(mdf['School'] == 'COLORADO MINES') & (mdf['Season'] == 'Cross Country')]
csm_f = fdf.loc[(fdf['School'] == 'COLORADO MINES') & (fdf['Season'] == 'Cross Country')]

In [32]:
csm_m
csm_f

,Name,Athlete ID,Grade,Academic_Year,School,Conference,Meet_ID,Meet_Name,Meet_Start,Meet_End,Year,Season,Event,Mark,Place,Prelim/Final,Time,Distance
7363,ZOE BAKER,6907869,JR,3,COLORADO MINES,Rocky Mountain AC,14511,2020 RMAC Cross Country Championships,"Oct 24, 2020","Oct 24, 2020",2020,Cross Country,6K,21:11.5,1.0,NaN,1271.5,6000.0000
7370,ZOE BAKER,6907869,JR,3,COLORADO MINES,Rocky Mountain AC,14554,NCAA Division II Cross Country Championships,"Nov 23, 2019","Nov 23, 2019",2019,Cross Country,6K,20:46.7,19.0,NaN,1246.7,6000.0000
7371,ZOE BAKER,6907869,JR,3,COLORADO MINES,Rocky Mountain AC,16015,NCAA Division II South Central Region Cross Co...,"Nov 9, 2019","Nov 9, 2019",2019,Cross Country,6K,21:38.0,5.0,NaN,1298.0,6000.0000
7372,ZOE BAKER,6907869,JR,3,COLORADO MINES,Rocky Mountain AC,14913,RMAC Cross Country Championships,"Oct 26, 2019","Oct 26, 2019",2019,Cross Country,6K,20:48.7,9.0,NaN,1248.7,6000.0000
7373,ZOE BAKER,6907869,JR,3,COLORADO MINES,Rocky Mountain AC,58709,Capital Cross Challenge,"Sep 28, 2019","Sep 28, 2019",2019,Cross Country,6K,20:51.4,8.0,NaN,1251.4,6000.0000
7374,ZOE BAKER,6907869,JR,3,COLORADO MINES,Rocky Mountain AC,64887,12th Annual UCCS Rust-Buster - 2019,"Sep 7, 2019","Sep 7, 2019",2019,Cross Country,5K,17:47.7,1.0,NaN,1067.7,5000.0000
7376,ZOE BAKER,6907869,JR,3,COLORADO MINES,Rocky Mountain AC,16713,NCAA Division II Cross Country Championships,"Dec 1, 2018","Dec 1, 2018",2018,Cross Country,6K,24:03.5,68.0,NaN,1443.5,6000.0000
7377,ZOE BAKER,6907869,JR,3,COLORADO MINES,Rocky Mountain AC,15218,NCAA Division II South Central Region Cross Co...,"Nov 17, 2018","Nov 17, 2018",2018,Cross Country,6K,22:16.5,13.0,NaN,1336.5,6000.0000
7378,ZOE BAKER,6907869,JR,3,COLORADO MINES,Rocky Mountain AC,63001,RMAC Cross Country Championships,"Nov 3, 2018","Nov 3, 2018",2018,Cross Country,6K,22:27.0,14.0,NaN,1347.0,6000.0000
7379,ZOE BAKER,6907869,JR,3,COLORADO MINES,Rocky Mountain AC,16302,SRU - Pre-Nationals,"Oct 20, 2018","Oct 20, 2018",2018,Cross Country,6K,22:53.9,8.0,NaN,1373.9,6000.0000


AttributeError: 'DataFrame' object has no attribute 'len'